In [1]:
import pandas as pd
from src.ConectorBD import *

In [2]:
import pandasql
from pandasql import sqldf

In [3]:
import koinenlp

In [61]:
# %pip list | grep cltk

cltk                               0.1.121
Note: you may need to restart the kernel to use updated packages.


In [62]:
# %pip install cltk==0.1.121 --force-reinstall

In [4]:
import nltk

In [5]:
import cltk

In [6]:
# This is the import of the right part of the CLTK library
from cltk.corpus.utils.importer import CorpusImporter

In [7]:
# Let's get some Greek corpora, too

my_greek_downloader = CorpusImporter('greek')
my_greek_downloader.import_corpus('greek_models_cltk')
my_greek_downloader.list_corpora

['greek_software_tlgu',
 'greek_text_perseus',
 'phi7',
 'tlg',
 'greek_proper_names_cltk',
 'greek_models_cltk',
 'greek_treebank_perseus',
 'greek_treebank_gorman',
 'greek_lexica_perseus',
 'greek_training_set_sentence_cltk',
 'greek_word2vec_cltk',
 'greek_text_lacus_curtius',
 'greek_text_first1kgreek',
 'greek_text_tesserae']

In [8]:
# corpus_importer = CorpusImporter('hebrew')

In [9]:
# corpus_importer.list_corpora

['hebrew_text_sefaria']

In [ ]:
# AT

In [8]:
column_names = ['book_nr', 'chapter_nr', 'verse_nr', 'part_of_speech', 'morphology', 'wd_sep_strongs']
lxx_source = pd.read_fwf('/media/s273524038/linux_bkp/Software/almeida-semantica/lxxm_utf8_v4', colspecs='infer', 
                         header=None, names=column_names, widths=[2, 3, 3, 4, 9, 50])

In [9]:
lxx_source.shape

(475400, 6)

In [9]:
# Split the 'Address' column by comma and expand into new columns
df_split = lxx_source['wd_sep_strongs'].str.split(' ', expand=True)

# Assign names to the new columns (optional, if you know the number of splits)
df_split.columns = ['normalized_word', 'strongs']

lxx_source.drop(['wd_sep_strongs'], axis=1, inplace=True)

# Concatenate the new columns back to the original DataFrame
lxx_source = pd.concat([lxx_source, df_split], axis=1)

In [ ]:
# del lxx

In [10]:
lxx_source.head()

,book_nr,chapter_nr,verse_nr,part_of_speech,morphology,normalized_word,strongs
0,1,1,1,P-,--------,εν,1722
1,1,1,1,N-,----DSF-,αρχη,746
2,1,1,1,V-,3AAI-S--,εποιησεν,4160
3,1,1,1,RA,----NSM-,ο,3588
4,1,1,1,N-,----NSM-,θεος,2316


In [11]:
lxx_source["book_nr"].nunique()

39

In [11]:
lxx_agrupada_versos = sqldf("""
                            select 
                                book_nr
                                , chapter_nr
                                , verse_nr
                                , group_concat(normalized_word, ' ') as txt
                            FROM 
                                lxx_source
                            group by
                                book_nr
                                , chapter_nr
                                , verse_nr            
                            """)

In [ ]:
# lxx_agrupada_versos.head()

In [ ]:
# NT

In [12]:
sbl_bd = '/media/s273524038/linux_bkp/Software/almeida-semantica/db/sbl.db'

In [19]:
# sql_sbl = "SELECT nr_sq_livro, chapter, verse, word, word_nr FROM words"

In [13]:
sql_sbl = "SELECT * FROM words"

In [15]:
# sql_sbl = "SELECT * FROM bible_fts"

In [14]:
gnt_df = pd.read_sql_query(sql_sbl, ConectorBD(sbl_bd).con)

In [16]:
gnt_df.head()

,book,chapter,verse,pos,morph,lemma,strongs,word,punct,word_nr,nr_sq_livro
0,Matt,1,1,N-,----NSF-,βίβλος,00976,Βίβλος,,1,40
1,Matt,1,1,N-,----GSF-,γένεσις,01078,γενέσεως,,2,40
2,Matt,1,1,N-,----GSM-,Ἰησοῦς,02424,Ἰησοῦ,,3,40
3,Matt,1,1,N-,----GSM-,Χριστός,05547,χριστοῦ,,4,40
4,Matt,1,1,N-,----GSM-,υἱός,05207,υἱοῦ,,5,40


In [18]:
# # del gnt_normal_txt
# gnt_df = sqldf("select * from gnt_df")

In [19]:
# new_col_values = [] 

# # sem normalização (só tira as quebras)
# for i, row in gnt_df.iterrows():
#     new_col_values.append(row['content'].strip())
    
# gnt_df['content'] = new_col_values

In [20]:
# new_col_values = [] 

# # criando coluna com texto 'normalizado'
# for i, row in gnt_df.iterrows():
#     new_col_values.append(koinenlp.normalize(row['content'].strip()))

# gnt_df['normalized_text'] = new_col_values

In [ ]:
# gnt_df.head()

In [82]:
# gnt_df.shape

(7927, 5)

In [22]:
# gnt_df['chapter'] = pd.to_numeric(gnt_df['chapter'], errors='coerce')
# gnt_df['verse'] = pd.to_numeric(gnt_df['verse'], errors='coerce')

In [16]:
print(gnt_df.dtypes)

book           object
chapter         int64
verse           int64
pos            object
morph          object
lemma          object
strongs        object
word           object
punct          object
word_nr         int64
nr_sq_livro     int64
dtype: object


In [87]:
gnt_df["book"].unique()

array(['Matt', 'Mark', 'Luke', 'John', 'Acts', 'Rom', '1Cor', '2Cor',
       'Gal', 'Eph', 'Phil', 'Col', '1Thess', '2Thess', '1Tim', '2Tim',
       'Titus', 'Phlm', 'Heb', 'Jas', '1Pet', '2Pet', '1John', '2John',
       '3John', 'Jude', 'Rev'], dtype=object)

In [49]:
# a partir daqui busca expressão GNT na LXX
# GNT só precisa normalize, pq já tem a coluna lemma


# nesse caso, vida e eterna é a mesma morfologia das palavras 
# que aparece na oração sacerdotal de João 17.3 e Daniel 12.2 (pelo lemma não acha)

gnt_df.query("book == 'John' and chapter == 17 and verse == 2")["word"].values[13:15]

array(['ζωὴν', 'αἰώνιον'], dtype=object)

In [61]:
gnt_df.query("book == 'Rom' and chapter == 8 and verse == 20")['word'].values

array(['τη γαρ ματαιοτητι η κτισισ υπεταγη ουχ εκουσα αλλα δια τον υποταξαντα ⸀επι ελπιδι'],
      dtype=object)

In [33]:
gnt_vs = [koinenlp.normalize(wd) for wd in 
                  gnt_df.query("book == 'Matt' and chapter == 3 and verse == 11")["lemma"].values]

In [34]:
gnt_vs

['εγω',
 'μεν',
 'συ',
 'βαπτιζω',
 'εν',
 'υδωρ',
 'εισ',
 'μετανοια',
 'ο',
 'δε',
 'οπισω',
 'εγω',
 'ερχομαι',
 'ισχυροσ',
 'εγω',
 'ειμι',
 'οσ',
 'ου',
 'ειμι',
 'ικανοσ',
 'ο',
 'υποδημα',
 'βασταζω',
 'αυτοσ',
 'συ',
 'βαπτιζω',
 'εν',
 'πνευμα',
 'αγιοσ',
 'και',
 'πυρ']

In [35]:
gnt_wd = gnt_vs[3]

In [39]:
gnt_wd = "βαπτω"

In [40]:
print(gnt_wd)

βαπτω


In [44]:
gnt_wd1, gnt_wd2 = gnt_vs[13], gnt_vs[14]

In [45]:
print(gnt_wd1, gnt_wd2)

ζωην αιωνιον


In [41]:
# busca lista de palavras em qualquer posição na lxx (normalizadas) 

ocorrencias_exp_lxx = sqldf("""
                            select * 
                            from lxx_agrupada_versos 
                            where 
                                txt like '%{}%' 
                            """.format(gnt_wd))

In [42]:
ocorrencias_exp_lxx.head()

,book_nr,chapter_nr,verse_nr,txt


In [ ]:
# a partir daqui buscar palavra/expressão da LXX no GNT
# LXX só precisa lemmatizer 



In [22]:
# eu vivo eternamente
lxx_exp = " ".join(lxx_source.query("book_nr == 5 and chapter_nr == 32 and verse_nr == 40")["normalized_word"][15:20])

In [8]:
# vida eterna 
lxx_exp = " ".join(lxx_source.query("book_nr == 27 and chapter_nr == 12 and verse_nr == 2")["normalized_word"][10:12])

In [50]:
# vaidade (última palavra de Eclesiastes 1.2)
lxx_exp = lxx_source.query("book_nr == 21 and chapter_nr == 1 and verse_nr == 2")["normalized_word"].values[-1]

In [55]:
lxx_source.query("book_nr == 21 and chapter_nr == 1 and verse_nr == 2")["normalized_word"].values[-1]

'ματαιοτης'

In [51]:
lxx_exp

'ματαιοτης'

In [22]:
lst_palavras_exp_lxx = lxx_exp.split(" ")

In [52]:
from cltk.stem.lemma import LemmaReplacer
# from cltk.corpus.utils.formatter import cltk_normalize

In [ ]:
# sentence = cltk_normalize(sentence)  # can help when using certain texts

In [53]:
lemmatizer = LemmaReplacer('greek')

In [54]:
lemmatized_sentence = lemmatizer.lemmatize(lxx_exp)

In [55]:
lemmatized_sentence 

['ματαιοτης']

In [57]:
lxx_wd1 = lemmatized_sentence[0]

In [61]:
# busca lista de palavras em qualquer posição no GNT (normalizadas) 
ocorrencias_exp_gnt = sqldf("""
                            select * 
                            from gnt_df 
                            where 
                                word like '%{}%' 
                            """.format(lxx_wd1))

In [ ]:
ocorrencias_exp_gnt = sqldf("""
                            select * 
                            from gnt_df 
                            where 
                                normalized_text like '%{}%' 
                            OR  normalized_text like '%{}%'
                            """.format(lxx_wd1, lxx_wd2))

In [63]:
len(ocorrencias_exp_gnt)

3

In [62]:
ocorrencias_exp_gnt.head()

,book,chapter,verse,pos,morph,lemma,strongs,word,punct,word_nr,nr_sq_livro


In [61]:
# gnt_df_gk_wd = pd.read_sql_query("SELECT * FROM words WHERE strongs like '3776'", ConectorBD(gnt).con)

In [11]:
# gnt_grouped_df = pd.read_sql_query("""
#                            SELECT
#                                 nr_sq_livro
#                                 , chapter
#                                 , verse
#                                 , group_concat(word, ' ') as words
#                                 , group_concat(lemma, ' ') as lemmas
#                                 , group_concat(strongs, ' ') as strongs
#                             FROM 
#                                 words
#                             group by
#                                 nr_sq_livro
#                                 , chapter
#                                 , verse""", ConectorBD(gnt).con)

In [63]:
gnt_df_gk_wd.head()

,book,chapter,verse,pos,morph,lemma,strongs,word,punct,word_nr,nr_sq_livro


In [12]:
# palavra_grega = koinenlp.normalize(strongs_grk_df['lemma'][1])
# import re
# palavra_grega = koinenlp.normalize(gnt_df.loc[(gnt_df['nr_sq_livro'] == 45) & (gnt_df['chapter'] == 10) & (gnt_df['verse'] == 9)].values[2][5])
# print(palavra_grega)
# df_query = lxx_df["verse_text"].str.contains(palavra_grega, flags=re.IGNORECASE, regex=True)
# df_query.value_counts()


ομολογεω


verse_text
False    30607
Name: count, dtype: int64